In [1]:
# prevent random files being included in dataset
!rm -rf `find -type d -name .ipynb_checkpoints`

In [2]:
import papermill as pm
import mlflow
import torch
from utils import md5_dir, set_seed
from torch.utils.data import DataLoader
import tqdm

from loss_functions import kd_loss
from datasets import TrainImageNetDataset

import torch
import torch.nn.functional as F

# if using pretrained model
from torchvision.models import ResNet50_Weights

/root/miniconda3/envs/mlrc_2023/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


/root/miniconda3/envs/mlrc_2023/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/root/miniconda3/envs/mlrc_2023/lib/python3.11/site-packages/torchvision/image.so: undefined symbol: _ZNK3c107SymBool10guard_boolEPKcl'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [3]:
set_seed(42)
torch.cuda.empty_cache()

In [4]:
# Default Parameters
run_id = "f13020b4e2304bd7837e10d17c6ea8ea"
train_data_path = "../data/ImageNet/ILSVRC/Data/CLS-LOC/train/"
# test_data_path = "data/ImageNet/ILSVRC/Data/CLS-LOC/val/"

train_data_labels_path = "../data/ImageNet/LOC_train_solution.csv"
# test_data_labels_path = "data/ImageNet/LOC_val_solution.csv"

label_mapping_path = "../data/ImageNet/LOC_synset_mapping.txt"

resnet50_weights = ResNet50_Weights.DEFAULT

preprocess = resnet50_weights.transforms()

device = "cuda" if torch.cuda.is_available() else "cpu"

In [5]:
# Parameters
run_id = "9b0844e061f440b69f900308457aef36"
train_data_path = "../data/ImageNet/ILSVRC/Data/CLS-LOC/train/"
train_data_labels_path = "../data/ImageNet/LOC_train_solution.csv"
label_mapping_path = "../data/ImageNet/LOC_synset_mapping.txt"


In [6]:
train_dataset = TrainImageNetDataset(train_data_path, train_data_labels_path, label_mapping_path, preprocess)

# test_dataset = TestImageNetDataset(test_data_path, test_data_labels_path, label_mapping_path, preprocess)

train_dataloader = DataLoader(train_dataset, batch_size=72, shuffle=True)
# test_dataloader = DataLoader(test_dataset, batch_size=80, shuffle=False)

../data/ImageNet/LOC_synset_mapping.txt


In [7]:
# if using pretrained model
from torchvision.models import resnet50, ResNet50_Weights, resnet18

resnet50_pretrained_weights = ResNet50_Weights.DEFAULT

teacher = resnet50(weights=resnet50_pretrained_weights)

student = resnet18(weights=None)
independent_student = resnet18(weights=None)

epochs = 100
lr = 0.001

In [8]:
def train_student(student, teacher, train_dataloader, criterion, optimizer, epochs, device):
    """
    - student: The smaller, untrained model that uses the teacher's output as an additional label
    - teacher: The pretrained model used to help the student model learn
    - train_dataloader: Dataloader for training data
    - criterion: The loss function
    - optimizer: The optimization algorithm
    - epochs: Number of training epochs
    - device: Device to run training
    """
    teacher.eval()
    teacher.to(device)
    student.train()
    student.to(device)
    
    for epoch in tqdm.tqdm(range(epochs)):
        running_loss = 0.0

        for inputs, labels in tqdm.tqdm(train_dataloader):
            inputs, labels = inputs.to(device), labels.to(device)
            labels = F.one_hot(labels, num_classes=1000).float()

            # Zero the gradients 
            optimizer.zero_grad()

            teacher_predictions = teacher(inputs)
            student_predictions = student(inputs)

            loss = criterion(student_predictions, labels, teacher_predictions, 0.5, 0.5)

            loss.backward()

            optimizer.step()

            running_loss += loss.item()
            break

        average_loss = running_loss / len(train_dataloader)
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {average_loss:.4f}')

        # save training loss in mlflow
        with mlflow.start_run(run_id=run_id) as run:
            mlflow.log_metric("student_training_loss", average_loss)

    with mlflow.start_run(run_id=run_id) as run:
        mlflow.pytorch.log_model(
            pytorch_model=teacher.to("cpu"),
            artifact_path="teacher",
        )

        mlflow.pytorch.log_model(
            pytorch_model=student.to("cpu"),
            artifact_path="student"
        )
        
        


optimizer = torch.optim.Adam(student.parameters(), lr)
train_student(student, teacher, train_dataloader, kd_loss, optimizer, epochs, device)

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/17794 [00:00<?, ?it/s]

  0%|          | 0/17794 [00:01<?, ?it/s]


  1%|          | 1/100 [00:01<01:48,  1.09s/it]

Epoch [1/100], Loss: 0.0003


...

  0%|          | 0/17794 [00:00<?, ?it/s]


100%|██████████| 100/100 [00:55<00:00,  1.80it/s]

100%|██████████| 100/100 [00:55<00:00,  1.81it/s]

Epoch [100/100], Loss: 0.0002


/root/miniconda3/envs/mlrc_2023/lib/python3.11/site-packages/_distutils_hack/__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
/root/miniconda3/envs/mlrc_2023/lib/python3.11/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [9]:
def train_independent_student(independent_student, train_dataloader, criterion, optimizer, epochs, device):
    """
    - teacher: The pretrained model used to help the student model learn
    - student: The smaller, untrained model that uses the teacher's output as an additional label
    - criterion: The loss function
    - optimizer: The optimization algorithm
    - epochs: Number of training epochs
    - device: Device to run training
    """
    independent_student.train()
    independent_student.to(device)
    
    for epoch in tqdm.tqdm(range(epochs)):
        running_loss = 0.0

        for inputs, labels in tqdm.tqdm(train_dataloader):
            inputs, labels = inputs.to(device), labels.to(device)
            labels = F.one_hot(labels, num_classes=1000).float()

            # Zero the gradients 
            optimizer.zero_grad()

            independent_student_predictions = independent_student(inputs)

            loss = criterion(independent_student_predictions, labels)

            loss.backward()

            optimizer.step()

            running_loss += loss.item()
            break

        average_loss = running_loss / len(train_dataloader)
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {average_loss:.4f}')


        # save training loss in mlflow
        with mlflow.start_run(run_id=run_id) as run:
            mlflow.log_metric("independent_student_training_loss", average_loss)

    with mlflow.start_run(run_id=run_id) as run:
        mlflow.pytorch.log_model(
            pytorch_model=independent_student.to("cpu"),
            artifact_path="independent_student"
        )

optimizer = torch.optim.Adam(independent_student.parameters(), lr)
criterion = torch.nn.CrossEntropyLoss()
train_independent_student(independent_student, train_dataloader, criterion, optimizer, epochs, device)

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/17794 [00:00<?, ?it/s]

  0%|          | 0/17794 [00:00<?, ?it/s]


  1%|          | 1/100 [00:00<00:40,  2.45it/s]

Epoch [1/100], Loss: 0.0004


...

  0%|          | 0/17794 [00:00<?, ?it/s]

  0%|          | 0/17794 [00:00<?, ?it/s]


100%|██████████| 100/100 [00:41<00:00,  2.51it/s]

100%|██████████| 100/100 [00:41<00:00,  2.41it/s]

Epoch [100/100], Loss: 0.0004
